In [5]:

"""
Inspired by code created by Diego De Lazzari
Modified for Python 3 by Aungshuman Zaman

"""
from selenium import webdriver
#from bs4 import BeautifulSoup # For HTML parsing
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
import pandas as pd # For converting results to a dataframe and bar chart plots
from selenium.webdriver.common import action_chains, keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np
import pickle
import re
import csv
import os.path
#from collections import OrderedDict
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

def init_driver():
    ''' Initialize chrome driver'''

    chrome_options = webdriver.ChromeOptions()

    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--profile-directory=Default')
    chrome_options.add_argument("--incognito")
    chrome_options.add_argument("--disable-plugins-discovery")
    chrome_options.add_argument("--start-maximized")
    #browser = webdriver.Chrome(driver, chrome_options=chrome_options)
    browser = webdriver.Chrome(chrome_options=chrome_options)
    #browser = webdriver.Chrome()

    return browser

##############################################################################

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

###############################################################################

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
###############################################################################

def get_pause():
    return np.random.choice(range(4,6))

###############################################################################
#utility function to get csv file from pickle.
def get_csv(pickle_obj):  ####&&&& Don' use it for new files
    my_dict = load_obj(pickle_obj)
    csv_filename = 'mycsvfile.csv'
    if os.path.isfile(csv_filename):
        print('File already exists! Please rename it.')
        return
    with open(csv_filename, 'w') as f:  # Just use 'w' mode in 3.x
        writer = csv.writer(f)

        for k,v in my_dict.items():
            if len(v) == 6:
                new_dict = {}
                new_dict['job_id'] = k
                new_dict['rating'] = v[0]
                new_dict['position'] = v[1]
                new_dict['company_name'] = v[2]
                new_dict['salary'] = v[3]
                new_dict['link'] = v[4]
                new_dict['description'] = v[5]  #[x.decode('ascii') for x in v[5]]
                writer.writerow(new_dict.values())


###############################################################################
def get_csv2(pickle_obj): ####&&&& Don' use it for new files
    my_dict = load_obj(pickle_obj)
    csv_filename = 'mycsvfile2.csv'
    if os.path.isfile(csv_filename):
        print('File already exists! Please rename it.')
        return
    with open(csv_filename, 'w') as f:  # Just use 'w' mode in 3.x
        writer = csv.writer(f)
        fieldnames = ['job_id','rating','position','company_name','salary','link',\
        'description','hq_city','hq_state_code','size','industry']

        for k,v in my_dict.items():
            if len(v) == 10:
                new_dict = {}
                new_dict['job_id'] = k
                new_dict['rating'] = v[0]
                new_dict['position'] = v[1]
                new_dict['company_name'] = v[2]
                new_dict['salary'] = v[3]
                new_dict['link'] = v[4]
                new_dict['description'] = v[5]  #[x.decode('ascii') for x in v[5]]
                new_dict['hq_city'] = v[6]
                new_dict['hq_state_code'] = v[7]
                new_dict['size'] = v[8]
                new_dict['industry'] = v[9]

                #writer.writerow(new_dict.values())
                writer.writerow([new_dict[i] for i in fieldnames]) #order preserved

##############################################################################

def get_csv3(pickle_obj): ####&&&&
    my_dict = load_obj(pickle_obj)
    csv_filename = 'mycsvfile3.csv'
    if os.path.isfile(csv_filename):
        print('File already exists! Please rename it.')
        return

    with open(csv_filename, 'w') as f:  # Just use 'w' mode in 3.x
        writer = csv.writer(f)
        fieldnames = ['job_id','rating', 'position', 'company', 'job_city', 'job_state_code',\
         'sal_low', 'sal_high', 'link','description','hq_city','hq_state_code','size','industry']

        for k,v in my_dict.items():
            if len(v) == 13:
                new_dict = {}
                new_dict['job_id'] = k
                new_dict['rating'] = v[0]
                new_dict['position'] = v[1]
                new_dict['company'] = v[2]
                new_dict['job_city'] = v[3]
                new_dict['job_state_code'] = v[4]
                new_dict['sal_low'] = v[5]
                new_dict['sal_high'] = v[6]
                new_dict['link'] = v[7]
                new_dict['description'] = [x.decode('ascii') for x in v[8]]
                #print(type(v[8]))
                new_dict['hq_city'] = v[9]
                new_dict['hq_state_code'] = v[10]
                new_dict['size'] = v[11]
                new_dict['industry'] = v[12]

                #writer.writerow(new_dict.values())
                writer.writerow([new_dict[i] for i in fieldnames]) #order preserved

#######



def searchJobs(browser, jobName, city=None, jobDict = None, link=None):
    '''Scrape for job listing'''

    ####&&&&
    #q = input('Shall we scrape? (y/n)\n') #q = raw_input('Shall we scrape? (y/n)')

    #if q=='y': ####&&&&
    if True:

        print('hello')
        job = browser.find_element_by_id("sc.keyword") #job title, keywords, or company
        
        location = browser.find_element_by_id("sc.location") #location search
        sleep(3)
        job.send_keys(jobName)  #type in job name in search
        sleep(2)
        #location form is already populated.
        location.clear()
        # can also execute JavaScript to clear it
        #browser.execute_script("arguments[0].value = ''", location)
        location.send_keys(city) #type in location name in search

        sleep(2)
        browser.find_element_by_class_name('gd-btn-mkt').click()

        sleep(5)
        

        # Find brief description


        for i in range(20): #20  ####&&&&
            try:
                # Extract useful classes
                jobPosting =browser.find_elements_by_class_name('jl')
                sleep(get_pause())

                # Create a job Dictionary. Every job in glassDoor has a unique data-id.
                # data-id should be used as key for the dictionary
                #create a map of 2-tuple. 2-tuple => data-id and selenium webElement.
                jobTuple = map(lambda a: (a.get_attribute('data-id'), a), jobPosting)

                # Filter picks out only those data-ids that are not in jobDict.keys()
                newPost = list(filter(lambda b: b[0] not in jobDict.keys(),jobTuple) ) #list of 2-tuple

                #If there are new posts, update job dict and link list
                if newPost != []:

                    # process the tuple
                    #example of a[1].text ->
                    #"3.7\nData Scientist, Analytics\nEtsy – Brooklyn, NY\n$114k-$167k  (Glassdoor Est.)\nWe're Hiring"
                    #tuple structure ('job_id',['rating','position','company','salary'])
                    #jobData = list(map(lambda a: (a[0],a[1].text.encode("utf8")./
                        #split('\n')[0:4]),newPost))
                    #jobData = list(map(lambda a: (a[0],a[1].text.split('\n')[0:4]),newPost))
                    #jobData = list(map(do_stuff, newPost)) ####&&&&
                    # do_stuff returns many misplaced entries.
                    #do_new_stuff uses regex to minimize bad data, it also splits up entries into more columns
                    # new tuple structure ('job_id',[rating, position, company, job_city, job_state_code, sal_low, sal_high])
                    print('starting do_new_stuff')
                    jobData = list(map(do_new_stuff, newPost))
                    print("I'm out of do_new_stuff.")

                    # Update job dictionary;
                    # Convert tuple to dictionary. structure ('job_id',['rating',...]) -> {'job_id':['rating',...]}
                    print('updating jobDict')
                    tmp = dict((a[0],a[1]) for a in jobData)
                    print('tmp created')
                    jobDict.update(tmp) #add a new entry with unique key job_id
                    # finally find the links:
                    link_lst = list(map(lambda c: (c[0],c[1].find_element_by_tag_name('a').\
                        get_attribute('href')), newPost))
                    #add the link to job dict
                    print('Adding to link')
                    tmp = [jobDict[c[0]].append(c[1]) for c in link_lst]
                    # update link list. This will be used in get_data part.
                    link += link_lst


                browser.find_element_by_class_name('next').click() #next page
                try:
                    #browser.find_element_by_class_name('xBtn').click() #pop-up
                    browser.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/div').click() #pop up
                except:
                    pass

            except Exception as e:
                #pass
                print(type(e),e)

    return jobDict, link

###############################################################################

def text_cleaner(text):
    '''
    This function just cleans up the raw html so that I can look at it.
    Inputs: a URL to investigate
    Outputs: Cleaned text only
    '''
    print('starting text_cleaner')
    stopws = set(stopwords.words("english"))
    #print('initialized stopws')

    lines = (line.strip() for line in text.splitlines()) # break into lines
    #lines = [line.strip() for line in text.splitlines()]

    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    #chunks = [phrase.strip() for line in lines for phrase in line.split("  ")]

    def chunk_space(chunk):
        chunk_out = chunk + ' ' # Need to fix spacing issue
        return chunk_out

    #print('Going for text!')
    text = ''.join(chunk_space(chunk) for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line


    # Now clean out all of the unicode junk (this line works great!!!)
    #print('cleaning out unicode junc from text!')
    try:
        text = text.decode('unicode_escape').encode('ascii', 'ignore') # Need this as some websites aren't formatted
    except:                                                            # in a way that this works, can occasionally throw
        return                                                         # an exception

    #print('getting rid of non-words from text!')
    text = re.sub(b"[^a-zA-Z.+3]",b" ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
                                                # Also include + for C++

    #print('make text lower case!')
    text = text.lower()  # Go to lower case

    #print('split text!')
    text = text.split()  #  and split them apart

    #print('removing stop words!')
    text = [w for w in text if not w in stopws]


    #print('set of text')
    text = list(set(text)) # Last, just get the set of these. Ignore counts
                           # we are just looking at whether a term existed or not on the website

    #print("We are done! Let's return it!")
    return text


##############################################################################
def string_from_text(pattern, tmp_txt):
    lst  = tmp_txt.split('\n')
    for x in lst:
        if x.find(pattern) != -1:
            return x.replace(pattern, ' ')

##############################################################################

def do_stuff(a):
    return (a[0],a[1].text.split('\n')[0:4])

##############################################################################

def do_new_stuff(a):
    print("I'm in do_new_stuff")
    if len(a) ==0:
        print('object is empty')

    tmp = a[1].text
    print(temp)
    raw_rating = re.findall('\d\.\d',tmp )
    print('raw_rating = ',raw_rating)
    if len(raw_rating)==1:
        rating =raw_rating[0]
    else:
        rating = ''
    raw_sal_range = re.findall('\d+k',tmp )
    print('raw_sal_range = ',raw_sal_range)
    if len(raw_sal_range)==2:
        sal_low = int(raw_sal_range[0].replace('k',''))
        sal_high = int(raw_sal_range[1].replace('k',''))
    else:
        sal_low = np.nan
        sal_high = np.nan
    raw_company = re.findall('.+–.+,.+',tmp)
    print('raw_company = ',raw_company)
    if len(raw_company)==1:
        tt = raw_company[0].split('–')
        company = tt[0].strip()
        job_city = tt[1].split(',')[0].strip()
        job_state_code = tt[1].split(',')[1].strip()
    else:
        company = ''
        job_city = ''
        job_state_code = ''
    raw_position = re.findall('(.+sci.+|.+ana.+|.+eng.+)',tmp.lower())
    print('raw_position = ',raw_position)
    if len(raw_position)==1:
        position = raw_position[0]
    else:
        position = tmp.split('\n')[1].lower()
    #return (a[0],tmp[0:4])
    print('Will go out of do_new_stuff.')
    return (a[0],[rating, position, company, job_city, job_state_code, sal_low, sal_high])

##############################################################################


In [3]:
from selenium import webdriver
#from bs4 import BeautifulSoup # For HTML parsing
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
import pandas as pd # For converting results to a dataframe and bar chart plots
from selenium.webdriver.common import action_chains, keys
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import sys
import re
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [10]:
# 1- Load existing dictionary. Check for initial dictionary.
# If empty initialize

try:
	jobDict = load_obj('glassDoorDict')
	link =    load_obj('glassDoorlink')
except:
	save_obj([], 'glassDoorlink')
	save_obj({}, 'glassDoorDict')

	jobDict = load_obj('glassDoorDict')
	link =    load_obj('glassDoorlink')

print('len(jobDict) = '+str(len(jobDict))+ ', len(link) = '+str(len(link)))

len(jobDict) = 1897, len(link) = 1396


In [4]:
# 2- Choose what you want to do:
#    get_link => Scraping for links and brief data,
#    get_data => Scraping for detailed data.


get_link = True ####&&&&
#get_link = False


website = "https://www.glassdoor.com/index.htm"
browser = webdriver.Firefox(executable_path=r'C:\Software\geckodriver-v0.25.0-win64\geckodriver.exe')
browser.get(website)
browser.find_element_by_css_selector('.gd-btn-locked-transparent').click()
username = browser.find_element_by_css_selector(".minh-modal > div:nth-child(1) > div:nth-child(5) > form:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(2) > input:nth-child(1)")
password = browser.find_element_by_css_selector("div.mt-xsm:nth-child(4) > div:nth-child(1) > div:nth-child(2) > input:nth-child(1)")
username.send_keys("tayyibah.rasheed@okstate.edu")
password.send_keys("GRazta123$")
browser.find_element_by_css_selector("div.mt-std:nth-child(5) > div:nth-child(1) > button:nth-child(1)").click()

In [181]:
# 4- Scrape for links and brief data

if get_link :
	iter_num = 0
	while iter_num <3: # default 1 ####&&&&
		print('Starting iteration number {}'.format(iter_num))
		sleep(get_pause())
		browser.get(website)

		# Initialize cities and jobs

		jobName_lst = ['Data Scientist', 'Data Analyst','Data Engineer']
		jobName = np.random.choice(jobName_lst)
		#jobName = 'Data Scientist' ####&&&&

		city_lst = ['San Jose','New York','San Francisco','Detroit','Washington','Austin','Boston','Seattle','Chicago','Los Angeles',' ']
		city = np.random.choice(city_lst)
		#city = ' '  ####&&&&

		print('jobName = '+jobName+ ', city = '+city)

		# search for jobs (short description)
		try:
			# jobDict structure {'job_id':['rating','position','company','salary']}
			update_jobDict, update_link = searchJobs(browser, jobName, city, jobDict, link)
			sleep(get_pause())
		except Exception as e:
			print(type(e),e)
			sys.exit("Error message")
			#pass


		print('len(update_jobDict) = '+str(len(update_jobDict))+ ', len(update_link) = '+str(len(update_link)))

		# save dictionary and link

		save_obj(update_jobDict, 'glassDoorDict')
		save_obj(update_link, 'glassDoorlink')

		iter_num += 1

	browser.close()

Starting iteration number 0
jobName = Data Scientist, city =  
hello
starting do_new_stuff
I'm in do_new_stuff
raw_rating =  ['3.8']
raw_sal_range =  []
raw_company =  []
raw_position =  ['data scientist', 'cape canaveral, fl']
Will go out of do_new_stuff.
I'm in do_new_stuff
raw_rating =  ['4.5']
raw_sal_range =  []
raw_company =  []
raw_position =  ['data scientist']
Will go out of do_new_stuff.
I'm in do_new_stuff
raw_rating =  ['3.6']
raw_sal_range =  []
raw_company =  []
raw_position =  ['data scientist']
Will go out of do_new_stuff.
I'm in do_new_stuff
raw_rating =  []
raw_sal_range =  []
raw_company =  []
raw_position =  ['data analyst']
Will go out of do_new_stuff.
I'm in do_new_stuff
raw_rating =  ['3.9']
raw_sal_range =  []
raw_company =  []
raw_position =  ['data scientist']
Will go out of do_new_stuff.
I'm in do_new_stuff
raw_rating =  ['3.2']
raw_sal_range =  []
raw_company =  []
raw_position =  ['jr. data scientist']
Will go out of do_new_stuff.
I'm in do_new_stuff
raw_ra

In [196]:
website = "https://www.glassdoor.com/index.htm"
browser = webdriver.Firefox(executable_path=r'C:\Software\geckodriver-v0.25.0-win64\geckodriver.exe')
browser.get(website)
browser.find_element_by_css_selector('.gd-btn-locked-transparent').click()
username = browser.find_element_by_css_selector(".minh-modal > div:nth-child(1) > div:nth-child(5) > form:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(2) > input:nth-child(1)")
password = browser.find_element_by_css_selector("div.mt-xsm:nth-child(4) > div:nth-child(1) > div:nth-child(2) > input:nth-child(1)")
username.send_keys("tayyibah.rasheed@okstate.edu")
password.send_keys("GRazta123$")
browser.find_element_by_css_selector("div.mt-std:nth-child(5) > div:nth-child(1) > button:nth-child(1)").click()

In [ ]:
get_data = True
if get_data:

	print('len(link) = '+str(len(link)))
	while len(link) > 200: # originally 0, a hard coded solution for when only bad links are left.
	#for i in range(250): # debugging	####&&&&

		try:
			rnd_job = np.random.choice(range(len(link)))
			#print(rnd_job)
			ids = link[rnd_job][0]
			page = link[rnd_job][1]
			#print(ids)
			#print(page)

			browser.get(page)
			#print(browser)
			sleep(3)

			# Extract text
			desc_list = browser.find_element_by_xpath('//*[@id="JobDescriptionContainer"]/div[1]').text
			#print('desc_list '+ str(type(desc_list)))
			description = text_cleaner(desc_list)
			#description = desc_list
			#print('description '+ str(type(description)))

			# jobDict structure {'job_id':['rating','position','company','salary','descr']}
			jobDict[ids].append(description)


			#Additional information about company (size, revenue, industry)
			sleep(2)
			try:
				browser.find_element_by_xpath('//*[@class="jobDetailsInfoWrap"]/header/div/div/div[2]/span').click()
				tmp_txt = browser.find_element_by_id('EmpBasicInfo').text

				hq_city = string_from_text('Headquarters', tmp_txt).split(',')[0].strip()
				#print('hq_city = ',hq_city)
				jobDict[ids].append(hq_city)
				#print(' 1 = ',)
				hq_state_code = string_from_text('Headquarters', tmp_txt).split(',')[0].strip()
				#print('hq_state_code = ',hq_state_code)
				jobDict[ids].append(hq_state_code)
				#print(' 2 = ',)
				#size_low = int(re.findall('\d+',string_from_text('Size',tmp_txt))[0])
				size = re.findall('\d+',string_from_text('Size',tmp_txt))
				#print('size = ', size)
				#size_high = int(re.findall('\d+',string_from_text('Size',tmp_txt))[1])
				#print(' = ',)
				jobDict[ids].append(size)
				#print(' 3 = ',)
				#jobDict[ids].append(size_low)
				#jobDict[ids].append(size_high)
				industry = string_from_text('Industry',tmp_txt)
				#print('industry = ',industry)

				jobDict[ids].append(industry)
				#print(' 4 = ',)
				#jobDict[ids].append(revenue)


			except Exception as e:
				print(type(e),e)

			#remove links already used

			dummy=link.pop(rnd_job)

			# if everything is fine, save
			#print("Going to save data!!")
			save_obj(jobDict, 'glassDoorDict')
			save_obj(link, 'glassDoorlink')

			print('Scraped successfully ' + ids)

			sleep(get_pause())
		except:
			print( str(ids) + ' is not working! Sleep for 6 seconds and retry')
			print( 'Still missing ' + str(len(link)) + ' links' )
			sleep(6)
	print('Done for now!! len(link) = '+str(len(link)))
	browser.close()

len(link) = 1894
starting text_cleaner
Scraped successfully 3372277973
starting text_cleaner
Scraped successfully 3277580140
starting text_cleaner
Scraped successfully 3363055482
starting text_cleaner
Scraped successfully 3320766370
starting text_cleaner
Scraped successfully 3290650232
starting text_cleaner
Scraped successfully 3374342143
starting text_cleaner
Scraped successfully 3343805917
starting text_cleaner
Scraped successfully 3359021883
starting text_cleaner
Scraped successfully 3334521649
starting text_cleaner
Scraped successfully 3361681197
starting text_cleaner
Scraped successfully 3378599427
starting text_cleaner
Scraped successfully 3203658495
starting text_cleaner
Scraped successfully 3349334324
starting text_cleaner
Scraped successfully 3328663903
starting text_cleaner
<class 'selenium.common.exceptions.NoSuchElementException'> Message: Unable to locate element: //*[@class="jobDetailsInfoWrap"]/header/div/div/div[2]/span

Scraped successfully 3286024103
starting text_cle

C:\Users\Tayyibah Kauser\Anaconda3\lib\site-packages\ipykernel_launcher.py:273: DeprecationWarning: invalid escape sequence '\ '


Scraped successfully 3314068841
starting text_cleaner
Scraped successfully 3384313298
starting text_cleaner
Scraped successfully 3368507422
starting text_cleaner
Scraped successfully 3272415482
starting text_cleaner
Scraped successfully 3289655136
starting text_cleaner
Scraped successfully 3291343494
starting text_cleaner
Scraped successfully 3003860377
starting text_cleaner
Scraped successfully 3375268777
starting text_cleaner
Scraped successfully 3306817399
starting text_cleaner
Scraped successfully 3378032137
starting text_cleaner
Scraped successfully 3364691231
starting text_cleaner
Scraped successfully 3378032136
starting text_cleaner
Scraped successfully 3357212147
starting text_cleaner
Scraped successfully 3207258830
starting text_cleaner
Scraped successfully 3379061371
starting text_cleaner
Scraped successfully 3380373458
starting text_cleaner
Scraped successfully 3257853325
starting text_cleaner
Scraped successfully 3296757431
starting text_cleaner
Scraped successfully 3157221

KeyboardInterrupt: 

In [20]:

my_dict = load_obj(pickle_obj)
csv_filename = 'mycsvfile2.csv'
if os.path.isfile(csv_filename):
    print('File already exists! Please rename it.')
    return

with open(csv_filename, 'w') as f: 
   # Just use 'w' mode in 3.x
    writer = csv.writer(f)
    print('y')
    fieldnames = ['job_id','rating', 'position', 'company', 'job_city', 'job_state_code',\
     'sal_low', 'sal_high', 'link','description','hq_city','hq_state_code','size','industry']

    for k,v in my_dict.items():
        if len(v) == 13:
            new_dict = {}
            new_dict['job_id'] = k
            new_dict['rating'] = v[0]
            new_dict['position'] = v[1]
            new_dict['company'] = v[2]
            new_dict['job_city'] = v[3]
            new_dict['job_state_code'] = v[4]
            new_dict['sal_low'] = v[5]
            new_dict['sal_high'] = v[6]
            new_dict['link'] = v[7]
            new_dict['description'] = v[8]
            #print(type(v[8]))
            new_dict['hq_city'] = v[9]
            new_dict['hq_state_code'] = v[10]
            new_dict['size'] = v[11]
            new_dict['industry'] = v[12]

            #writer.writerow(new_dict.values())
            writer.writerow([new_dict[i] for i in fieldnames]) #order preserved

#

In [22]:
my_dict = load_obj('glassDoorDict')
csv_filename = 'mycsvfile2.csv'

In [27]:
for k in my_dict.keys():
    print(len(my_dict[k]))

14
13
13
13
13
13
13
13
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
9
13
13
13
13
13
13
11
13
13
13
13
13
13
13
13
13
13
13
13
13
13
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
12
13
13
13
13
13
13
13
13
13
9
13
13
13
13
13
13
13
13
13
13
9
13
13
13
9
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
9
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
12
13
13
13
13
13
13
9
12
13
13
13
13
13
12
13
13
13
13
13
13
13
13
13
13
13
12
13
13
13
9
13
13
13
13
13
13
13
13
13
13
13
13
12
13
13
13
13
9
13
13
13
13
13
13
13
13
13
12
13
13
13
13
13
13
13
13
12
13
13
13
13
12
9
13
13
13
13
13
13
12
13
13
8
8
13
13
13
13
13
13
13
13
12
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13


In [29]:
my_dict['3381578119']

['3.9',
 'data engineer',
 'Seniorlink',
 'Boston',
 'MA',
 64,
 89,
 'https://www.glassdoor.com/partner/jobListing.htm?pos=101&ao=680969&s=149&guid=0000016de58a38219b7e47f6471ee345&src=GD_JOB_AD&t=SRFJ&extid=4&exst=OL&ist=&ast=OL&vt=w&slr=true&cs=1_833d2534&cb=1571514104766&jobListingId=3381578119',
 'Position Summary:\n\n\nThe Data Engineer will be a key player in Seniorlink’s Product & Technologies Organization (P&T). You will be the primary person building out the data integration framework capability within the ‘DataHub’ architecture. The DataHub uses a modern cloud based distributed computing architecture that utilizes modern technologies such as AWS S3, AWS EMR, Apache Spark, Redshift etc. You will work on the foundational infrastructure components within this architecture to expand the capability and make the system more robust. You will collaborate with senior engineers to design and engineer a robust, scalable integration framework that can transform the variety of incoming d

3381578119


KeyError: 'job_id'

In [35]:
new_dict

{'job_id': '3361635360',
 'rating': '4.2',
 'position': 'data engineer',
 'company': 'Keller Williams',
 'job_city': 'Austin',
 'job_state_code': 'TX',
 'sal_low': 57,
 'sal_high': 79,
 'link': 'https://www.glassdoor.com/partner/jobListing.htm?pos=128&ao=6101&s=58&guid=0000016de59855da9a2f9fc34270d041&src=GD_JOB_AD&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&cs=1_24f9fe92&cb=1571515030030&jobListingId=3361635360',
 'description': 'What youll be called: Data Engineer\n\nWhere youll work: KWRI HeadquartersAustin, TX\n\nNamed a Happiest Company to Work for in 2019; one of the Best Places to Work in Austin, TX; and featured on the Training Magazine Training 125 list seven times, Keller Williams Realty International (KWRI) thrives within a creative and collaborative culture where transforming the real estate industry through technology is our primary goal.\n\nKW Technology is the foremost provider of real estate solutions, offering the most comprehensive end-to-end portfolio of products,

In [38]:
new_dict = {'job_id' : [], 
            'rating' : [], 
            'position' : [], 
            'company' : [],
            'job_city' : [],
            'job_state_code' : [],
            'sal_low' : [],
            'sal_high' : [],
            'link' : [],
            'description' : [],
            'hq_city' : [],
            'hq_state_code' : [],
            'size' : [],
            'industry' : []}


In [39]:
for k,v in my_dict.items():
    print(k)
    if len(v)>= 13:
        new_dict['job_id'].append(k)
        new_dict['rating'].append(v[0])
        new_dict['position'].append(v[1])
        new_dict['company'].append(v[2])
        new_dict['job_city'].append(v[3])
        new_dict['job_state_code'].append(v[4])
        new_dict['sal_low'].append(v[5])
        new_dict['sal_high'].append(v[6])
        new_dict['link'].append(v[7])
        new_dict['description'].append(v[8])
        #print(type(v[8]))
        new_dict['hq_city'].append(v[9])
        new_dict['hq_state_code'].append(v[10])
        new_dict['size'].append(v[11])
        new_dict['industry'].append(v[12])

3381578119
3320321158
3284284037
3298422474
3352031827
3196505395
3388905001
3374099933
3379908362
3356833111
3350001434
3386703038
3378009255
3354376776
3180307018
3376562653
3254097283
3359023976
3238036556
3296285380
3350511635
3356138485
3356602294
3328266779
3322135751
3391450036
3365068364
3205983875
3353556402
3318225561
3042963959
3328903440
3253403496
3308918595
3379951822
3358722522
3373840950
3328124307
3335457872
3351864423
3365674970
3219890595
3279849002
3246517577
3385750069
3374690234
3178235988
2994117434
3346481137
3353579290
3374589739
3329734179
3326030111
3325134847
3322135794
3383653153
3300388779
3374670113
2725035780
3227655543
3325280156
3272706711
3323871719
3265401585
3364762246
3361416053
3384363812
3302841314
3363451785
3358528772
3365248211
3344983747
3332214881
3280026843
3311524017
3391745761
3388114590
3317344619
3181397648
3093497333
3272792918
3349197179
3348319164
3377840501
3379783823
3388498166
3280026844
3386482594
3364762219
3391910866
3389794716

In [40]:
new_dict

{'job_id': ['3381578119',
  '3320321158',
  '3284284037',
  '3298422474',
  '3352031827',
  '3196505395',
  '3388905001',
  '3374099933',
  '3356833111',
  '3350001434',
  '3386703038',
  '3378009255',
  '3354376776',
  '3180307018',
  '3376562653',
  '3254097283',
  '3359023976',
  '3238036556',
  '3296285380',
  '3350511635',
  '3356138485',
  '3356602294',
  '3328266779',
  '3322135751',
  '3391450036',
  '3365068364',
  '3205983875',
  '3353556402',
  '3318225561',
  '3328903440',
  '3253403496',
  '3308918595',
  '3379951822',
  '3358722522',
  '3373840950',
  '3335457872',
  '3351864423',
  '3365674970',
  '3219890595',
  '3279849002',
  '3246517577',
  '3385750069',
  '3374690234',
  '3178235988',
  '2994117434',
  '3346481137',
  '3353579290',
  '3374589739',
  '3329734179',
  '3325134847',
  '3322135794',
  '3383653153',
  '3300388779',
  '3374670113',
  '2725035780',
  '3227655543',
  '3325280156',
  '3272706711',
  '3323871719',
  '3265401585',
  '3364762246',
  '3361416053'

In [44]:
df =  pd.DataFrame(new_dict)

In [46]:
df.to_excel('E:/OSU - MSBAN/3rd Sem/MSIS 5600/glassdoor.xlsx')